In [ ]:
import gcsfs
import zarr
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio
from rasterio.features import rasterize
from skimage.measure import block_reduce
import time
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed
from satextractor.utils import tqdm_joblib
import pandas as pd
import geopandas as gpd
from oxeo.water.metrics import metrics
import gcsfs
fs = gcsfs.GCSFileSystem(project='oxeo-main')
from joblib import Parallel, delayed
from satextractor.utils import tqdm_joblib

#import fsspec
#fsspec.asyn.iothread[0] = None
#fsspec.asyn.loop[0] = None

In [ ]:
# Create datset 
# dam 
#   | patch 1
#      | data , labels
#   | patch 2

# Lo que tengo que hacer es descargar un tile completo e recorrerlo sacando patches
# o directamente con zarr ir descargando los parches con slices

In [ ]:
dams = ["harangi", "hemavathy", "kabini", "krishnaraja_sagar"]
patches = []
for d in dams:
    patches.extend(fs.ls(f'oxeo-water/eo/{d}/'))
patches

In [ ]:
import os
def store_revisit_patches(z_arr, revisit,patch_size, output, source_arr='data'):
    ori_arr = z_arr[revisit]
    
    for i in range(ori_arr.shape[-2]//patch_size):
        for j in range(ori_arr.shape[-1]//patch_size):
            patch_start_i = i*patch_size
            patch_end_i = patch_start_i + patch_size
            patch_start_j = j*patch_size
            patch_end_j = patch_start_j + patch_size
           
            patch_arr = ori_arr[..., patch_start_i:patch_end_i, patch_start_j:patch_end_j]
            out_dir = f"{output}/patch_{revisit}_{patch_start_i}_{patch_start_j}"
            os.makedirs(out_dir, exist_ok=True)
            np.save(f"{out_dir}/{source_arr}",patch_arr, allow_pickle=False)
         

In [ ]:
from multiprocessing import Pool
from itertools import repeat

def split_in_patches(tile_path, patch_size=100, dst_dir="../data", source_arr="data"):

    if source_arr == "data":
        data_type = np.uint16
    else:
        data_type = np.int8
    z_arr = zarr.open_array(f"gs://{tile_path}/sentinel-2/{source_arr}", dtype=data_type)
    
    with tqdm_joblib(
            tqdm(
                desc="parallel getting patches.",
                total=z_arr.shape[0],
            ),
        ):
            masks = Parallel(n_jobs=-1, verbose=0, prefer="threads")(
                [
                    delayed(store_revisit_patches)(z_arr, revisit, 100, 
                        f"{dst_dir}/{tile_path}", source_arr
                    )
                    for revisit in range(z_arr.shape[0])
                ],
            )

    

In [ ]:
#for p in patches:
#    split_in_patches(p, source_arr="data")
    
#for p in patches:
#    split_in_patches(p, source_arr="weak_labels")

In [ ]:
from oxeo.water.datamodules.segmentation_datamodule import SegmentationDataModule

In [ ]:
train_dams = ["harangi","hemavathy","krishnaraja_sagar"]
val_dams = ["kabini"]

In [ ]:

import glob
train_dams = ["harangi","hemavathy","krishnaraja_sagar"]
val_dams = ["kabini"]
train_paths = []
for d in train_dams:
    train_paths.extend(glob.glob(f"/home/fran/repos/oxeo-water/data/oxeo-water/eo/{d}/*/*"))
val_paths = glob.glob(f"/home/fran/repos/oxeo-water/data/oxeo-water/eo/kabini/*/*")

In [ ]:
'/home/fran/repos/oxeo-water/data/oxeo-water/eo/harangi/32643_0_0/patch_30_0_100/'

In [ ]:
dm = SegmentationDataModule(num_workers=8, batch_size=256, train_paths=train_paths, val_paths=val_paths)

In [ ]:
dm.setup()

In [ ]:
np.load(f"{train_paths[0]}/data.npy").shape

In [ ]:
t = next(iter(dm.train_dataloader()))

In [ ]:
t['label'].shape

In [ ]:
for t in iter(dm.train_dataloader()):
    print(t['data'].shape)

In [ ]:
split_in_patches('gs://oxeo-water/eo/kabini/32643_0_0/sentinel-2', source_arr="weak_labels")

In [ ]:
!pip install lightning-bolts